<a href="https://colab.research.google.com/github/tutsilianna/Gift_Recommendation_System/blob/main/Gift_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Аннотация

Список литературы:
1. https://education.yandex.ru/handbook/ml/article/intro-recsys
2. Анализ текстовых данных https://habr.com/ru/companies/otus/articles/774498/
---
https://neurohive.io/ru/tutorial/sozdaem-rekomendatelnuju-sistemu-s-ispolzovaniem-biblioteki-fastai/

https://habr.com/ru/companies/skillfactory/articles/585182/   

https://neerc.ifmo.ru/wiki/index.php?title=%D0%A0%D0%B5%D0%BA%D0%BE%D0%BC%D0%B5%D0%BD%D0%B4%D0%B0%D1%82%D0%B5%D0%BB%D1%8C%D0%BD%D1%8B%D0%B5_%D1%81%D0%B8%D1%81%D1%82%D0%B5%D0%BC%D1%8B

https://vc.ru/u/1389654-machine-learning/592278-5-luchshih-generatorov-sinteticheskih-dannyh-na-python-i-kak-ih-ispolzovat-kogda-vam-ne-hvataet-dannyh

### Есть три варианта реализации:


1.  Content-Based Filtering (Фильтрация на основе контента)

    **Принцип:** Анализирует характеристики подарков и предпочтения пользователя (пол, возраст, интересы, хобби).
    
    **Данные:** Описания подарков, профили пользователей.
    
    **Алгоритмы:**TF-IDF, cosine similarity.
    
    **Пример:** Если пользователь увлекается фотографией, система порекомендует фотоаппарат, объектив или аксессуары для фотосъемки.

2. Collaborative Filtering (Коллаборативная фильтрация):

    **Принцип:** Анализирует историю покупок и оценок пользователей, чтобы найти похожих пользователей и рекомендовать подарки, которые понравились им.

    **Данные:** История покупок, рейтинги, отзывы.

    **Алгоритмы:** K-Nearest Neighbors, Matrix Factorization.

    **Пример:** Если пользователь А и пользователь Б имеют похожую историю покупок, и пользователь А купил книгу, которая понравилась пользователю Б, система порекомендует эту книгу другим пользователям с похожими предпочтениями.



3. Hybrid Approach (Гибридный подход):

    **Принцип:** Комбинирует content-based и collaborative filtering для повышения точности рекомендаций.

    **Пример:** Система использует content-based filtering, чтобы найти подарки, соответствующие интересам пользователя, а затем использует collaborative filtering, чтобы ранжировать эти подарки на основе предпочтений похожих пользователей.



* Дополнительные факторы:

    **Бюджет пользователя:** Система должна учитывать бюджет пользователя при рекомендации подарков.

    **Повод:** Подарки на день рождения, Новый год или свадьбу будут отличаться.

    **Социальная информация:** Информация о друзьях и семье пользователя в социальных сетях может помочь в подборе подарков.

**Входные параметры:** (опционально) пол, возраст, интересы/хобби - строковое описание увлечений человека.

**Данные для обучения:** даные о пользователе (входные параметры) + какие подарки он выбрал. Эти данные будут синтетическими

## Генерация анкет

In [ ]:
import random

# Список интересов
interests_list = ["спорт", "музыка", "кино", "книги", "игры", "кулинария",
                  "путешествия", "искусство", "технологии", "мода", "красота",
                  "шоппинг", "автомобили", "рыбалка", "охота", "финансы",
                  "образование", "здоровье", "саморазвитие", "домашние животные",
                  "садоводство", "фотография", "танцы", "йога", "волонтерство",
                  "рисование", "живопись", "скульптура", "фотография", "музыка",
                  "танцы", "театр", "кино", "писательство", "поэзия", "рукоделие",
                  "дизайн", "кулинария", "марки", "монеты", "комиксы", "антиквариат",
                  "игрушки", "видеоигры", "компьютерные игры", "мобильные игры",
                  "настольные игры", "азартные игры"]

# Функция для генерации данных пользователя
def generate_user():
    gender = random.choice(["male", "female"])
    age = random.randint(18, 75)
    num_interests = random.randint(2, 5)  # Количество интересов
    interests = random.sample(interests_list, num_interests)
    return f"{gender}\t{age}\t{','.join(interests)}"

# Генерация и запись данных
with open("users.txt", "w") as f:
    for _ in range(5000):
        user_data = generate_user()
        f.write(user_data + "\n")

print("Файл users.txt создан!")


Файл users.txt создан!


## Подключение репозитория, откуда берем данные

In [1]:
! git clone https://github.com/tutsilianna/Gift_Recommendation_System

Cloning into 'Gift_Recommendation_System'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 30 (delta 7), reused 12 (delta 1), pack-reused 0
Receiving objects: 100% (30/30), 11.57 MiB | 5.23 MiB/s, done.
Resolving deltas: 100% (7/7), done.


## Импорт необходимых библиотек

In [2]:
import re
import nltk
import pandas as pd
import numpy as np
from nltk import stem
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation
from gensim.models import Word2Vec
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer



nltk.download("stopwords")

mystem = Mystem()
russian_stopwords = stopwords.words("russian")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


## Чтение данных и предобработка

In [3]:
data = pd.read_csv('/content/Gift_Recommendation_System/data/raw/ozon_data_.csv')
data.head(2)

,название,описание,код товара,оценка,количество отзывов,цена,категория,подкатегория,подподкатегория,ссылка
0,Проектор для смартфона 2-го поколения своими р...,Проектор для смартфона (проектор для смартфона...,Артикул: 1320591182,3.3,13,808,Электроника,Телевизоры и видеотехника,Проекторы,https://www.ozon.ru/product/proektor-dlya-smar...
1,"Проектор 90411567, белый",3D увеличитель экрана телефона – это универсал...,Артикул: 619939085,4.4,205,300,Электроника,Телевизоры и видеотехника,Проекторы,https://www.ozon.ru/product/proektor-90411567-...


## **1. Подготовка данных:**



### **Предобработка данных:** обработка пропущенных значений, удаление дубликатов, преобразование столбца "код товара";    


In [4]:
data['подподкатегория'] = data['подподкатегория'].fillna('')

In [5]:
data.drop_duplicates(inplace=True)
data.dropna(inplace=True)

In [6]:
# удаление слова из "Код товара", удаление строк,
# где нет атрибута и преобразование столбца к численному типу

data['код товара'] = data['код товара'].str.rsplit().str[-1]
data = data.loc[data['код товара'].str.isdigit()]
data['код товара'] = data['код товара'].astype(np.int64)

<ipython-input-6-5c128f6b6fec>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['код товара'] = data['код товара'].astype(np.int64)


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9374 entries, 0 to 13099
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   название            9374 non-null   object 
 1   описание            9374 non-null   object 
 2   код товара          9374 non-null   int64  
 3   оценка              9374 non-null   float64
 4   количество отзывов  9374 non-null   int64  
 5   цена                9374 non-null   int64  
 6   категория           9374 non-null   object 
 7   подкатегория        9374 non-null   object 
 8   подподкатегория     9374 non-null   object 
 9   ссылка              9374 non-null   object 
dtypes: float64(1), int64(3), object(6)
memory usage: 805.6+ KB


In [8]:
data.loc[:, 'ключевые слова'] = data['категория'] + ', ' + data['подкатегория'] + ', ' + data['подкатегория']
data.drop(columns=['категория', 'подкатегория', 'подподкатегория'], axis=1, inplace=True)

<ipython-input-8-f89b99985585>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, 'ключевые слова'] = data['категория'] + ', ' + data['подкатегория'] + ', ' + data['подкатегория']
<ipython-input-8-f89b99985585>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.drop(columns=['категория', 'подкатегория', 'подподкатегория'], axis=1, inplace=True)


In [9]:
text_columns = [column for column in data.columns if data[column].dtype == 'object' and column != 'ссылка']
text_columns

['название', 'описание', 'ключевые слова']

### **Предобработка текста:** лемматизация и стемминг для полей "название", "описание" и "ключевые слова" (объединение "категории", "подкатегории" и "подподкатегории"), чтобы привести слова к базовой форме и улучшить поиск.


In [10]:
def preprocess_text(text):
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token not in russian_stopwords\
              and token != " " \
              and token.strip() not in punctuation]

    text = " ".join(tokens)

    return text

In [11]:
data_ = data.copy(deep=True)

In [12]:
data[text_columns] = data[text_columns].apply(lambda x: x.apply(preprocess_text))

In [13]:
data[text_columns].head(3)

,название,описание,ключевые слова
0,проектор смартфон 2 го поколение свой рука под...,проектор смартфон проектор смартфон свой рука ...,электроника телевизор видеотехника телевизор в...
1,проектор 90411567 белый,3d увеличитель экран телефон – это универсал...,электроника телевизор видеотехника телевизор в...
2,проектор 90411553,3d увеличительный экран ваш смартфон тренд тик...,электроника телевизор видеотехника телевизор в...


### **Векторизация данных:** TF-IDF и word2vec для преобразования текстовых данных ("название", "описание" и "ключевые слова") в числовые векторы.


In [14]:
data["текст"] = data[text_columns].apply(lambda x: " ".join(x), axis=1)
data.head(2)

,название,описание,код товара,оценка,количество отзывов,цена,ссылка,ключевые слова,текст
0,проектор смартфон 2 го поколение свой рука под...,проектор смартфон проектор смартфон свой рука ...,1320591182,3.3,13,808,https://www.ozon.ru/product/proektor-dlya-smar...,электроника телевизор видеотехника телевизор в...,проектор смартфон 2 го поколение свой рука под...
1,проектор 90411567 белый,3d увеличитель экран телефон – это универсал...,619939085,4.4,205,300,https://www.ozon.ru/product/proektor-90411567-...,электроника телевизор видеотехника телевизор в...,проектор 90411567 белый 3d увеличитель экран т...


#### TD-IDF

In [15]:
# Создание TF-IDF векторайзера
vectorizer = TfidfVectorizer(max_features=1000)  # Можно настроить max_features

# Обучение векторайзера и преобразование текста в векторы
tfidf_vectors = vectorizer.fit_transform(data["текст"])

# Преобразование в DataFrame
tfidf_df = pd.DataFrame(tfidf_vectors.toarray(), columns=vectorizer.get_feature_names_out())

# Объединение с исходным DataFrame
data_tfidf = pd.concat([data, tfidf_df], axis=1)

In [16]:
data_tfidf.head(2)

,название,описание,код товара,оценка,количество отзывов,цена,ссылка,ключевые слова,текст,10,...,эпоха,этап,это,эффект,эффективный,юный,являться,язык,японский,яркий
0,проектор смартфон 2 го поколение свой рука под...,проектор смартфон проектор смартфон свой рука ...,1.320591e+09,3.3,13.0,808.0,https://www.ozon.ru/product/proektor-dlya-smar...,электроника телевизор видеотехника телевизор в...,проектор смартфон 2 го поколение свой рука под...,0.0,...,0.0,0.0,0.092498,0.071953,0.039145,0.0,0.0,0.0,0.0,0.000000
1,проектор 90411567 белый,3d увеличитель экран телефон – это универсал...,6.199391e+08,4.4,205.0,300.0,https://www.ozon.ru/product/proektor-90411567-...,электроника телевизор видеотехника телевизор в...,проектор 90411567 белый 3d увеличитель экран т...,0.0,...,0.0,0.0,0.081970,0.079704,0.000000,0.0,0.0,0.0,0.0,0.127591


#### Word2Vec

In [17]:
# Подготовка текстов для Word2Vec
text_corpus = [text.split() for text in data["текст"]]

# Обучение Word2Vec модели
model = Word2Vec(text_corpus, vector_size=100, window=5, min_count=5, workers=4)  # Настройте параметры

# Получение векторов для каждого текста
word2vec_vectors = []
for text in text_corpus:
    word_vectors = [model.wv[word] for word in text if word in model.wv]
    if word_vectors:
        text_vector = np.mean(word_vectors, axis=0)
    else:
        text_vector = np.zeros(model.vector_size)
    word2vec_vectors.append(text_vector)

# Преобразование в DataFrame
word2vec_df = pd.DataFrame(word2vec_vectors)

# Объединение с исходным DataFrame
data_word2vec = pd.concat([data, word2vec_df], axis=1)

In [18]:
data_word2vec.head(2)

,название,описание,код товара,оценка,количество отзывов,цена,ссылка,ключевые слова,текст,0,...,90,91,92,93,94,95,96,97,98,99
0,проектор смартфон 2 го поколение свой рука под...,проектор смартфон проектор смартфон свой рука ...,1.320591e+09,3.3,13.0,808.0,https://www.ozon.ru/product/proektor-dlya-smar...,электроника телевизор видеотехника телевизор в...,проектор смартфон 2 го поколение свой рука под...,-0.054771,...,0.452048,-0.345536,-0.035106,0.248704,0.246775,0.060303,-0.395649,-0.266040,-0.030724,-0.393983
1,проектор 90411567 белый,3d увеличитель экран телефон – это универсал...,6.199391e+08,4.4,205.0,300.0,https://www.ozon.ru/product/proektor-90411567-...,электроника телевизор видеотехника телевизор в...,проектор 90411567 белый 3d увеличитель экран т...,0.006987,...,0.514755,-0.608177,-0.070561,0.137583,0.350719,0.091536,-0.479948,-0.199651,0.075290,-0.463852


### **Нормализация числовых данных:** Нормализовация данные о цене и количестве отзывов, чтобы они имели одинаковый масштаб.


In [19]:
normalize_column = [column for column in data.columns if data[column].dtype != 'object' and column != 'код товара']
data[normalize_column].head(2)

,оценка,количество отзывов,цена
0,3.3,13,808
1,4.4,205,300


In [20]:
scaler = MinMaxScaler().fit(data[normalize_column])

data_normalize = scaler.transform(data[normalize_column])
data[normalize_column] = pd.DataFrame(data_normalize, columns = normalize_column)

data[normalize_column].head(2)

,оценка,количество отзывов,цена
0,0.66,0.000330,0.000457
1,0.88,0.005199,0.000169


### **??? Кодирование категориальных данных:** Использовать one-hot encoding для категориальных данных, таких как пол и интересы.

## **2.  Выбор модели:**

* **Контент-базированная фильтрация:**
    * Вычислить сходство между векторами запроса пользователя (пол, возраст, интересы) и векторами товаров.
    * Для этого можно использовать косинусную меру или евклидово расстояние.
    * Рекомендуйте товары с наибольшим сходством.

* **Коллаборативная фильтрация:**
    * Если доступны данные о прошлых покупках или оценках пользователей, можно использовать алгоритмы SVD или ALS для выявления скрытых факторов, влияющих на выбор подарков.
    * На основе этих факторов можно прогнозировать, какие товары понравятся пользователю с заданным профилем.


In [24]:
from sklearn.metrics.pairwise import cosine_similarity

def get_user_query_vector(user_query, vectorizer):
    interest_vectors = []
    for interest in user_query["интересы"]:
        interest_vector = vectorizer.transform([interest]).toarray()
        interest_vectors.append(interest_vector)
    average_vector = np.mean(interest_vectors, axis=0)
    # Преобразование среднего вектора в пространство TF-IDF
    # average_vector_tfidf = vectorizer.transform(average_vector.reshape(1, -1)).toarray()
    return average_vector#_tfidf

def get_recommendations(similarities, data, n=10):
    """
    Получение рекомендаций на основе сходства.
    """
    # Индексы товаров, отсортированные по убыванию сходства
    top_indices = similarities.argsort()[-n:]
    # Получение данных о рекомендованных товарах
    recommendations = data_.iloc[top_indices]
    return recommendations

# Пример запроса пользователя
user_query = {
    # "пол": "мужской",
    # "возраст": 30,
    "интересы": ["дети", "вязание", "рукоделие"]
}

# Векторизация запроса пользователя
user_query_vector = get_user_query_vector(user_query, vectorizer)

# Вычисление сходства (косинусное сходство)
# similarities = cosine_similarity(user_query_vector.reshape(1, -1), tfidf_df)
similarities = []
for i in range(len(tfidf_df)):
    row_mean = np.mean(tfidf_df.iloc[i].values, axis=0)
    user_mean = np.mean(user_query_vector, axis=1)
    similarity = cosine_similarity(np.array([row_mean]).reshape(1, -1), user_mean.reshape(1, -1))
    similarities.append(similarity[0][0])

similarities_array = np.array(similarities)


# Получение рекомендаций
recommendations = get_recommendations(similarities_array, data)

# Вывод рекомендаций
# recommendations["ссылка"].values
recommendations[["название", "цена"]]

,название,цена
4371,Почти серьёзно | Никулин Ю.,698
4372,ЗИЯЮЩИЕ ВЫСОТЫ 2023 | Зиновьев Александр Алекс...,964
4373,"Дневник Гуантанамо | Симс Ларри, Слахи Мохамме...",662
4374,Россия победит. Жириновский В.В. | Жириновский...,523
4375,"Нечисть Швеции: обитатели кладбищ, лесов и полей",1450
4377,LED ZEPPELIN. Самая полная биография,1731
4378,"Б91 Хроника СЕРДЦА: Дневники, размышления, сюж...",1742
4379,Черчилль. Биография | Гилберт Мартин,1771
4353,"Что-то не так с Гэлвинами. Идеальная семья, ра...",149
13099,"Калимба музыкальный инструмент 17 нот, Kalimba...",920


## **3.  Гибридный подход:** (пока этого не будет)

* Комбинируйте контент-базированную и коллаборативную фильтрацию для большей точности рекомендаций.



## **4. Ранжирование и фильтрация:**

* **Ранжирование:** Отсортируйте рекомендации по убыванию сходства или прогнозируемого рейтинга.
* **Фильтрация:** Примените фильтры по цене, категории, подкатегории и т.д., чтобы сузить список рекомендаций в соответствии с предпочтениями пользователя.



## **5.  Дополнительные функции:**

* **Персонализация:** Используйте информацию о прошлых взаимодействиях пользователя с системой для повышения релевантности рекомендаций.
* **Разнообразие:** Стремитесь к разнообразию в рекомендациях, избегая предлагать слишком похожие товары.
* **Объяснение:** Отобразите пользователю, почему ему были рекомендованы те или иные товары.